# 14.32 final paper - step 3, regressions

Start with simple DD, then add complexity.

In [1]:
import pandas as pd
import numpy as np
import microdf as mdf
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from statsmodels.stats.weightstats import DescrStatsW
from stargazer.stargazer import Stargazer

## Data

In [90]:
cps = pd.read_csv('data/cps.csv')

## Slices for different regressions

In [99]:
cps_c = cps[~cps.days_sc.isna()]
cps_c_lwe = cps_c[~cps_c.lwe.isna()]
cps_c_hours = cps_c[~cps_c.hours.isna()]

## Common

In [86]:
cps.columns

Index(['const', 'YEAR', 'SERIAL', 'MONTH', 'HWTFINL', 'CPSID', 'COUNTY',
       'FAMINC', 'w', 'CPSIDP', 'age', 'SEX', 'MARST', 'NCHILD', 'NCHLT5',
       'EMPSTAT', 'hours', 'EARNWEEK', 'POPESTIMATE2019',
       'school_closure_date', 'female', 'cps_end_date', 'cps_start_date',
       'unemp', 'emp', 'lf', 'has_k6', 'days_sc', 'post', 'post_has_k6',
       'other_faminc', 'emp_of_lf', 'lwe', 'days_since_2000', 'apr2020',
       'apr2020_has_k6', 'days_sc_has_k6', 'age2', 'apr2020_has_k6_female',
       'apr2020_female', 'has_k6_female', 'days_sc_has_k6_female',
       'days_sc_female'],
      dtype='object')

In [54]:
OUTCOMES = ['emp', 'hours', 'lwe']

## DD

First version: only compare February and April

In [55]:
dd0 = cps[(cps.YEAR == 2020) & cps.MONTH.isin([2, 4])][
    OUTCOMES + ['apr2020', 'has_k6', 'apr2020_has_k6', 'w', 'const']]

In [56]:
weighted_stats = DescrStatsW(dd0.lwe[~dd0.lwe.isna()], 
                             weights=dd0.w[~dd0.lwe.isna()], ddof=0)
weighted_stats.std_mean

0.00011043335582375969

In [57]:
sm.WLS(dd0.emp, dd0[['const', 'apr2020', 'has_k6', 'apr2020_has_k6']],
       weights=dd0.w, missing='drop').fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                    emp   R-squared:                       0.015
Model:                            WLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     420.4
Date:                Sat, 30 May 2020   Prob (F-statistic):          4.89e-271
Time:                        16:51:02   Log-Likelihood:                -56582.
No. Observations:               81467   AIC:                         1.132e+05
Df Residuals:                   81463   BIC:                         1.132e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.8009      0.003    278.364      0.000       0.795       0.807
apr2020           -0.1117      0.004    -27.397      0.000      -0.120      -0.104
has_k6             0.0028      0.004      0.647      0.518      -0.006       0.011
apr2020_has_k6     0.0112      0.006      1.845      0.065      -0.001       0.023
==============================================================================
Omnibus:                    13915.774   Durbin-Watson:                   1.903
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22558.412
Skew:                          -1.286   Prob(JB):                         0.00
Kurtosis:                       3.181   Cond. No.                         6.61
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [58]:
sm.WLS(dd0.hours, dd0[['const', 'apr2020', 'has_k6', 'apr2020_has_k6']],
       weights=dd0.w, missing='drop').fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  hours   R-squared:                       0.004
Model:                            WLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     88.97
Date:                Sat, 30 May 2020   Prob (F-statistic):           1.94e-57
Time:                        16:51:02   Log-Likelihood:            -2.3740e+05
No. Observations:               59107   AIC:                         4.748e+05
Df Residuals:                   59103   BIC:                         4.748e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             40.4259      0.089    452.550      0.000      40.251      40.601
apr2020           -1.6426      0.134    -12.301      0.000      -1.904      -1.381
has_k6            -0.1920      0.133     -1.447      0.148      -0.452       0.068
apr2020_has_k6     0.0949      0.198      0.480      0.631      -0.292       0.482
==============================================================================
Omnibus:                     7951.259   Durbin-Watson:                   1.945
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            99058.005
Skew:                           0.160   Prob(JB):                         0.00
Kurtosis:                       9.334   Cond. No.                         6.37
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [59]:
sm.WLS(dd0.lwe, dd0[['const', 'apr2020', 'has_k6', 'apr2020_has_k6']],
       weights=dd0.w, missing='drop').fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                    lwe   R-squared:                       0.002
Model:                            WLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     12.03
Date:                Sat, 30 May 2020   Prob (F-statistic):           7.34e-08
Time:                        16:51:03   Log-Likelihood:                -17711.
No. Observations:               14642   AIC:                         3.543e+04
Df Residuals:                   14638   BIC:                         3.546e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              6.7826      0.011    620.129      0.000       6.761       6.804
apr2020            0.0551      0.016      3.420      0.001       0.024       0.087
has_k6             0.0639      0.016      3.893      0.000       0.032       0.096
apr2020_has_k6    -0.0205      0.024     -0.851      0.395      -0.068       0.027
==============================================================================
Omnibus:                     6359.500   Durbin-Watson:                   1.812
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           101414.636
Skew:                          -1.669   Prob(JB):                         0.00
Kurtosis:                      15.454   Cond. No.                         6.38
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Simple with more history

Can do the same with `cps` instead of `dd0`, after dropping March 2020,
since treatment wasn't the same then (requires clustering).

In [92]:
cps_no_mar2020 = cps[~((cps.YEAR == 2020) & (cps.MONTH == 3))]

In [61]:
sm.WLS(cps_no_mar2020.lwe,
       cps_no_mar2020[['const', 'apr2020', 'has_k6', 'apr2020_has_k6']],
       weights=cps_no_mar2020.w, missing='drop').fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                    lwe   R-squared:                       0.001
Model:                            WLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     55.62
Date:                Sat, 30 May 2020   Prob (F-statistic):           6.27e-36
Time:                        16:51:08   Log-Likelihood:            -2.3253e+05
No. Observations:              189752   AIC:                         4.651e+05
Df Residuals:                  189748   BIC:                         4.651e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              6.7493      0.002   2886.611      0.000       6.745       6.754
apr2020            0.0885      0.012      7.413      0.000       0.065       0.112
has_k6             0.0212      0.003      6.114      0.000       0.014       0.028
apr2020_has_k6     0.0222      0.018      1.250      0.211      -0.013       0.057
==============================================================================
Omnibus:                   100453.189   Durbin-Watson:                   1.829
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3332784.734
Skew:                          -1.949   Prob(JB):                         0.00
Kurtosis:                      23.158   Cond. No.                         13.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [62]:
sm.WLS(cps_no_mar2020.hours,
       cps_no_mar2020[['const', 'apr2020', 'has_k6', 'apr2020_has_k6']],
       weights=cps_no_mar2020.w, missing='drop').fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  hours   R-squared:                       0.001
Model:                            WLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     247.0
Date:                Sat, 30 May 2020   Prob (F-statistic):          3.00e-160
Time:                        16:51:10   Log-Likelihood:            -3.1752e+06
No. Observations:              795760   AIC:                         6.350e+06
Df Residuals:                  795756   BIC:                         6.350e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             40.6667      0.018   2236.004      0.000      40.631      40.702
apr2020           -1.8834      0.095    -19.730      0.000      -2.070      -1.696
has_k6            -0.2011      0.027     -7.515      0.000      -0.254      -0.149
apr2020_has_k6     0.1040      0.141      0.739      0.460      -0.172       0.380
==============================================================================
Omnibus:                   129636.743   Durbin-Watson:                   1.944
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1593134.783
Skew:                           0.403   Prob(JB):                         0.00
Kurtosis:                       9.885   Cond. No.                         13.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [63]:
sm.WLS(cps_no_mar2020.emp,
       cps_no_mar2020[['const', 'apr2020', 'has_k6', 'apr2020_has_k6']],
       weights=cps_no_mar2020.w, missing='drop').fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                    emp   R-squared:                       0.003
Model:                            WLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     980.1
Date:                Sat, 30 May 2020   Prob (F-statistic):               0.00
Time:                        16:51:10   Log-Likelihood:            -6.4094e+05
No. Observations:             1031956   AIC:                         1.282e+06
Df Residuals:                 1031952   BIC:                         1.282e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.7975      0.001   1444.944      0.000       0.796       0.799
apr2020           -0.1083      0.003    -41.040      0.000      -0.113      -0.103
has_k6             0.0055      0.001      6.776      0.000       0.004       0.007
apr2020_has_k6     0.0084      0.004      2.159      0.031       0.001       0.016
==============================================================================
Omnibus:                   259810.840   Durbin-Watson:                   1.935
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           504690.806
Skew:                          -1.627   Prob(JB):                         0.00
Kurtosis:                       4.072   Cond. No.                         12.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### History plus more controls

In [64]:
sm.WLS(cps_no_mar2020.emp,
       cps_no_mar2020[['const', 'apr2020', 'has_k6', 'apr2020_has_k6',
                       'days_since_2000', 'female', 'age', 'age2']],
       weights=cps_no_mar2020.w, missing='drop').fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                    emp   R-squared:                       0.029
Model:                            WLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     4331.
Date:                Sat, 30 May 2020   Prob (F-statistic):               0.00
Time:                        16:51:15   Log-Likelihood:            -6.2747e+05
No. Observations:             1031956   AIC:                         1.255e+06
Df Residuals:                 1031948   BIC:                         1.255e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.5325      0.017     31.112      0.000       0.499       0.566
apr2020            -0.1143      0.003    -41.990      0.000      -0.120      -0.109
has_k6              0.0180      0.001     20.880      0.000       0.016       0.020
apr2020_has_k6      0.0077      0.004      1.984      0.047    9.37e-05       0.015
days_since_2000  1.645e-05   2.07e-06      7.951      0.000    1.24e-05    2.05e-05
female             -0.1277      0.001   -162.044      0.000      -0.129      -0.126
age                 0.0117      0.000     24.406      0.000       0.011       0.013
age2               -0.0002   6.01e-06    -25.811      0.000      -0.000      -0.000
==============================================================================
Omnibus:                   250281.028   Durbin-Watson:                   1.906
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           476108.655
Skew:                          -1.571   Prob(JB):                         0.00
Kurtosis:                       4.097   Cond. No.                     3.17e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.17e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [65]:
sm.WLS(cps_no_mar2020.lwe,
       cps_no_mar2020[['const', 'apr2020', 'has_k6', 'apr2020_has_k6',
                       'days_since_2000', 'female', 'age', 'age2']],
       weights=cps_no_mar2020.w, missing='drop').fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                    lwe   R-squared:                       0.060
Model:                            WLS   Adj. R-squared:                  0.060
Method:                 Least Squares   F-statistic:                     1718.
Date:                Sat, 30 May 2020   Prob (F-statistic):               0.00
Time:                        16:51:28   Log-Likelihood:            -2.2678e+05
No. Observations:              189752   AIC:                         4.536e+05
Df Residuals:                  189744   BIC:                         4.537e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               4.1264      0.072     57.620      0.000       3.986       4.267
apr2020             0.0394      0.012      3.276      0.001       0.016       0.063
has_k6             -0.0482      0.004    -13.263      0.000      -0.055      -0.041
apr2020_has_k6      0.0122      0.017      0.708      0.479      -0.022       0.046
days_since_2000     0.0001   8.65e-06     15.363      0.000       0.000       0.000
female             -0.2697      0.003    -81.789      0.000      -0.276      -0.263
age                 0.0861      0.002     42.635      0.000       0.082       0.090
age2               -0.0009   2.54e-05    -37.206      0.000      -0.001      -0.001
==============================================================================
Omnibus:                   107650.021   Durbin-Watson:                   1.809
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          4229802.395
Skew:                          -2.104   Prob(JB):                         0.00
Kurtosis:                      25.744   Cond. No.                     3.16e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.16e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [66]:
sm.WLS(cps_no_mar2020.hours,
       cps_no_mar2020[['const', 'apr2020', 'has_k6', 'apr2020_has_k6',
                       'days_since_2000', 'female', 'age', 'age2']],
       weights=cps_no_mar2020.w, missing='drop').fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  hours   R-squared:                       0.041
Model:                            WLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     4824.
Date:                Sat, 30 May 2020   Prob (F-statistic):               0.00
Time:                        16:51:36   Log-Likelihood:            -3.1591e+06
No. Observations:              795760   AIC:                         6.318e+06
Df Residuals:                  795752   BIC:                         6.318e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              33.9870      0.561     60.572      0.000      32.887      35.087
apr2020            -1.8906      0.097    -19.485      0.000      -2.081      -1.700
has_k6             -0.3127      0.028    -11.044      0.000      -0.368      -0.257
apr2020_has_k6     -0.0492      0.138     -0.357      0.721      -0.320       0.221
days_since_2000  9.282e-05   6.76e-05      1.374      0.170   -3.96e-05       0.000
female             -4.5066      0.026   -174.487      0.000      -4.557      -4.456
age                 0.3628      0.016     22.894      0.000       0.332       0.394
age2               -0.0038      0.000    -18.872      0.000      -0.004      -0.003
==============================================================================
Omnibus:                   132615.857   Durbin-Watson:                   1.912
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1666315.273
Skew:                           0.417   Prob(JB):                         0.00
Kurtosis:                      10.040   Cond. No.                     3.16e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.16e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Intensity DD with fixed effects and clustering

Only a few observations from the March CPS were in counties with closed schools. Any signal they might contribute would likely be offset by the need to cluster by county.

Instead, apply a different signal: days since school closure. This is effectively a DD where treatment varies in intensity. Pair this with county fixed effects and standard error clustering at the county level. We can now include March 2020 data.

### 

In [76]:
sm.WLS(cps_c.emp,
       cps_c[['const', 'days_sc', 'has_k6', 'days_sc_has_k6',
              'days_since_2000', 'female', 'age', 'age2']],
       weights=cps_c.w, missing='drop').fit(
    cov_type='cluster', cov_kwds={'groups': cps_c.COUNTY}).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                    emp   R-squared:                       0.028
Model:                            WLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     176.6
Date:                Sat, 30 May 2020   Prob (F-statistic):           1.56e-74
Time:                        16:57:38   Log-Likelihood:            -2.0829e+05
No. Observations:              363768   AIC:                         4.166e+05
Df Residuals:                  363760   BIC:                         4.167e+05
Df Model:                           7                                         
Covariance Type:              cluster                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.5635      0.051     10.945      0.000       0.463       0.664
days_sc            -0.0040      0.000    -12.818      0.000      -0.005      -0.003
has_k6              0.0102      0.004      2.433      0.015       0.002       0.018
days_sc_has_k6      0.0003      0.000      0.937      0.349      -0.000       0.001
days_since_2000  1.583e-05   6.44e-06      2.459      0.014    3.21e-06    2.84e-05
female             -0.1230      0.005    -26.937      0.000      -0.132      -0.114
age                 0.0107      0.002      6.038      0.000       0.007       0.014
age2               -0.0001   2.24e-05     -6.369      0.000      -0.000   -9.89e-05
==============================================================================
Omnibus:                    90393.867   Durbin-Watson:                   1.909
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           174320.124
Skew:                          -1.604   Prob(JB):                         0.00
Kurtosis:                       4.098   Cond. No.                     3.07e+05
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The condition number is large, 3.07e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [80]:
sm.WLS(cps_c_lwe.lwe,
       cps_c_lwe[['const', 'days_sc', 'has_k6', 'days_sc_has_k6',
                  'days_since_2000', 'female', 'age', 'age2']],
       weights=cps_c_lwe.w, missing='drop').fit(
    cov_type='cluster', cov_kwds={'groups': cps_c_lwe.COUNTY}).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                    lwe   R-squared:                       0.056
Model:                            WLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                     183.6
Date:                Sat, 30 May 2020   Prob (F-statistic):           8.69e-76
Time:                        17:07:23   Log-Likelihood:                -80472.
No. Observations:               68086   AIC:                         1.610e+05
Df Residuals:                   68078   BIC:                         1.610e+05
Df Model:                           7                                         
Covariance Type:              cluster                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               3.8944      0.168     23.127      0.000       3.564       4.224
days_sc             0.0008      0.001      1.118      0.263      -0.001       0.002
has_k6             -0.0827      0.014     -5.732      0.000      -0.111      -0.054
days_sc_has_k6      0.0006      0.001      0.531      0.595      -0.002       0.003
days_since_2000     0.0002   1.77e-05      8.695      0.000       0.000       0.000
female             -0.2427      0.010    -23.736      0.000      -0.263      -0.223
age                 0.0926      0.005     19.234      0.000       0.083       0.102
age2               -0.0010   5.92e-05    -17.177      0.000      -0.001      -0.001
==============================================================================
Omnibus:                    32995.634   Durbin-Watson:                   1.799
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           829105.403
Skew:                          -1.795   Prob(JB):                         0.00
Kurtosis:                      19.714   Cond. No.                     3.06e+05
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The condition number is large, 3.06e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [84]:
sm.WLS(cps_c_hours.hours,
       cps_c_hours[['const', 'days_sc', 'has_k6', 'days_sc_has_k6',
                    'days_since_2000', 'female', 'age', 'age2']],
       weights=cps_c_hours.w, missing='drop').fit(
    cov_type='cluster', cov_kwds={'groups': cps_c_hours.COUNTY}).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  hours   R-squared:                       0.036
Model:                            WLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     182.8
Date:                Sat, 30 May 2020   Prob (F-statistic):           1.17e-75
Time:                        17:08:46   Log-Likelihood:            -1.1120e+06
No. Observations:              283593   AIC:                         2.224e+06
Df Residuals:                  283585   BIC:                         2.224e+06
Df Model:                           7                                         
Covariance Type:              cluster                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              32.2384      1.345     23.977      0.000      29.603      34.874
days_sc            -0.0478      0.007     -7.321      0.000      -0.061      -0.035
has_k6             -0.5358      0.115     -4.654      0.000      -0.761      -0.310
days_sc_has_k6     -0.0086      0.009     -0.928      0.353      -0.027       0.010
days_since_2000     0.0001      0.000      0.838      0.402      -0.000       0.000
female             -4.0574      0.134    -30.308      0.000      -4.320      -3.795
age                 0.4268      0.051      8.304      0.000       0.326       0.527
age2               -0.0046      0.001     -7.129      0.000      -0.006      -0.003
==============================================================================
Omnibus:                    43614.343   Durbin-Watson:                   1.904
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           545657.578
Skew:                           0.334   Prob(JB):                         0.00
Kurtosis:                       9.763   Cond. No.                     3.07e+05
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The condition number is large, 3.07e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Diff-in-diff-in-diff

Add interactions by female and SES.

### Simple DIDID

In [93]:
sm.WLS(cps_no_mar2020.emp,
       cps_no_mar2020[['apr2020_has_k6_female',  # Beta of interest.
                       'apr2020_female', 'apr2020_has_k6', 'has_k6_female',
                       'const', 'apr2020', 'has_k6',
                       'days_since_2000', 'female', 'age', 'age2']],
       weights=cps_no_mar2020.w, missing='drop').fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                    emp   R-squared:                       0.034
Model:                            WLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     3648.
Date:                Sat, 30 May 2020   Prob (F-statistic):               0.00
Time:                        18:03:07   Log-Likelihood:            -6.2449e+05
No. Observations:             1031956   AIC:                         1.249e+06
Df Residuals:                 1031945   BIC:                         1.249e+06
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
apr2020_has_k6_female    -0.0193      0.008     -2.489      0.013      -0.034      -0.004
apr2020_female            0.0118      0.005      2.263      0.024       0.002       0.022
apr2020_has_k6            0.0173      0.006      3.116      0.002       0.006       0.028
has_k6_female            -0.1212      0.002    -75.095      0.000      -0.124      -0.118
const                     0.4985      0.017     29.196      0.000       0.465       0.532
apr2020                  -0.1200      0.004    -33.175      0.000      -0.127      -0.113
has_k6                    0.0810      0.001     67.471      0.000       0.079       0.083
days_since_2000        1.626e-05   2.06e-06      7.884      0.000    1.22e-05    2.03e-05
female                   -0.0724      0.001    -66.313      0.000      -0.075      -0.070
age                       0.0124      0.000     25.958      0.000       0.011       0.013
age2                     -0.0002      6e-06    -27.720      0.000      -0.000      -0.000
==============================================================================
Omnibus:                   245953.656   Durbin-Watson:                   1.905
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           463045.528
Skew:                          -1.552   Prob(JB):                         0.00
Kurtosis:                       4.065   Cond. No.                     3.17e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.17e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [95]:
sm.WLS(cps_no_mar2020.lwe,
       cps_no_mar2020[['apr2020_has_k6_female',  # Beta of interest.
                       'apr2020_female', 'apr2020_has_k6', 'has_k6_female',
                       'const', 'apr2020', 'has_k6',
                       'days_since_2000', 'female', 'age', 'age2']],
       weights=cps_no_mar2020.w, missing='drop').fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                    lwe   R-squared:                       0.067
Model:                            WLS   Adj. R-squared:                  0.067
Method:                 Least Squares   F-statistic:                     1359.
Date:                Sat, 30 May 2020   Prob (F-statistic):               0.00
Time:                        18:04:02   Log-Likelihood:            -2.2605e+05
No. Observations:              189752   AIC:                         4.521e+05
Df Residuals:                  189741   BIC:                         4.522e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
apr2020_has_k6_female     0.0155      0.035      0.449      0.653      -0.052       0.083
apr2020_female            0.0011      0.023      0.047      0.962      -0.044       0.047
apr2020_has_k6            0.0028      0.024      0.117      0.907      -0.044       0.050
has_k6_female            -0.2538      0.007    -37.741      0.000      -0.267      -0.241
const                     4.0502      0.071     56.751      0.000       3.910       4.190
apr2020                   0.0390      0.016      2.474      0.013       0.008       0.070
has_k6                    0.0751      0.005     15.399      0.000       0.066       0.085
days_since_2000           0.0001   8.61e-06     15.362      0.000       0.000       0.000
female                   -0.1535      0.005    -33.679      0.000      -0.162      -0.145
age                       0.0879      0.002     43.679      0.000       0.084       0.092
age2                     -0.0010   2.53e-05    -38.361      0.000      -0.001      -0.001
==============================================================================
Omnibus:                   108095.827   Durbin-Watson:                   1.809
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          4331267.490
Skew:                          -2.111   Prob(JB):                         0.00
Kurtosis:                      26.022   Cond. No.                     3.16e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.16e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [94]:
sm.WLS(cps_no_mar2020.hours,
       cps_no_mar2020[['apr2020_has_k6_female',  # Beta of interest.
                       'apr2020_female', 'apr2020_has_k6', 'has_k6_female',
                       'const', 'apr2020', 'has_k6',
                       'days_since_2000', 'female', 'age', 'age2']],
       weights=cps_no_mar2020.w, missing='drop').fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  hours   R-squared:                       0.045
Model:                            WLS   Adj. R-squared:                  0.045
Method:                 Least Squares   F-statistic:                     3726.
Date:                Sat, 30 May 2020   Prob (F-statistic):               0.00
Time:                        18:03:44   Log-Likelihood:            -3.1574e+06
No. Observations:              795760   AIC:                         6.315e+06
Df Residuals:                  795749   BIC:                         6.315e+06
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
apr2020_has_k6_female     0.1195      0.277      0.432      0.666      -0.422       0.661
apr2020_female            0.6957      0.188      3.693      0.000       0.326       1.065
apr2020_has_k6           -0.1648      0.188     -0.877      0.381      -0.533       0.204
has_k6_female            -2.9819      0.053    -56.643      0.000      -3.085      -2.879
const                    33.0947      0.560     59.082      0.000      31.997      34.193
apr2020                  -2.1978      0.127    -17.355      0.000      -2.446      -1.950
has_k6                    1.0927      0.038     29.076      0.000       1.019       1.166
days_since_2000        8.343e-05   6.74e-05      1.237      0.216   -4.87e-05       0.000
female                   -3.1406      0.036    -87.290      0.000      -3.211      -3.070
age                       0.3857      0.016     24.380      0.000       0.355       0.417
age2                     -0.0041      0.000    -20.543      0.000      -0.004      -0.004
==============================================================================
Omnibus:                   133845.840   Durbin-Watson:                   1.911
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1684374.862
Skew:                           0.426   Prob(JB):                         0.00
Kurtosis:                      10.076   Cond. No.                     3.16e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.16e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Intensity DIDID

In [104]:
sm.WLS(cps_c_lwe.lwe,
       cps_c_lwe[['days_sc_has_k6_female',  # Beta of interest.
                    'days_sc_female', 'days_sc_has_k6', 'has_k6_female',
                    'const', 'apr2020', 'has_k6',
                    'days_since_2000', 'female', 'age', 'age2']],
       weights=cps_c_lwe.w, missing='drop').fit(
    cov_type='cluster', cov_kwds={'groups': cps_c_lwe.COUNTY}).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                    lwe   R-squared:                       0.065
Model:                            WLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     190.6
Date:                Sat, 30 May 2020   Prob (F-statistic):           7.27e-87
Time:                        18:12:06   Log-Likelihood:                -80162.
No. Observations:               68086   AIC:                         1.603e+05
Df Residuals:                   68075   BIC:                         1.604e+05
Df Model:                          10                                         
Covariance Type:              cluster                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
days_sc_has_k6_female    -0.0005      0.002     -0.276      0.782      -0.004       0.003
days_sc_female            0.0011      0.001      0.931      0.352      -0.001       0.003
days_sc_has_k6            0.0009      0.002      0.576      0.564      -0.002       0.004
has_k6_female            -0.2771      0.015    -18.528      0.000      -0.306      -0.248
const                     3.7948      0.166     22.884      0.000       3.470       4.120
apr2020                   0.0057      0.030      0.191      0.849      -0.053       0.064
has_k6                    0.0534      0.016      3.333      0.001       0.022       0.085
days_since_2000           0.0002   1.74e-05      8.869      0.000       0.000       0.000
female                   -0.1225      0.010    -11.678      0.000      -0.143      -0.102
age                       0.0950      0.005     19.683      0.000       0.086       0.104
age2                     -0.0011   5.96e-05    -17.622      0.000      -0.001      -0.001
==============================================================================
Omnibus:                    33141.774   Durbin-Watson:                   1.800
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           851404.841
Skew:                          -1.799   Prob(JB):                         0.00
Kurtosis:                      19.946   Cond. No.                     3.07e+05
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The condition number is large, 3.07e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [103]:
sm.WLS(cps_c.emp,
       cps_c[['days_sc_has_k6_female',  # Beta of interest.
              'days_sc_female', 'days_sc_has_k6', 'has_k6_female',
              'const', 'apr2020', 'has_k6',
              'days_since_2000', 'female', 'age', 'age2']],
       weights=cps_c.w, missing='drop').fit(
    cov_type='cluster', cov_kwds={'groups': cps_c.COUNTY}).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                    emp   R-squared:                       0.034
Model:                            WLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     151.3
Date:                Sat, 30 May 2020   Prob (F-statistic):           3.80e-79
Time:                        18:09:37   Log-Likelihood:            -2.0718e+05
No. Observations:              363768   AIC:                         4.144e+05
Df Residuals:                  363757   BIC:                         4.145e+05
Df Model:                          10                                         
Covariance Type:              cluster                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
days_sc_has_k6_female    -0.0007      0.000     -1.540      0.124      -0.002       0.000
days_sc_female            0.0005      0.000      1.485      0.137      -0.000       0.001
days_sc_has_k6            0.0007      0.000      1.756      0.079   -8.47e-05       0.002
has_k6_female            -0.1223      0.008    -16.054      0.000      -0.137      -0.107
const                     0.5242      0.051     10.375      0.000       0.425       0.623
apr2020                  -0.1400      0.011    -12.427      0.000      -0.162      -0.118
has_k6                    0.0743      0.004     18.949      0.000       0.067       0.082
days_since_2000        1.574e-05    6.4e-06      2.460      0.014     3.2e-06    2.83e-05
female                   -0.0697      0.004    -16.382      0.000      -0.078      -0.061
age                       0.0117      0.002      6.611      0.000       0.008       0.015
age2                     -0.0002   2.23e-05     -7.019      0.000      -0.000      -0.000
==============================================================================
Omnibus:                    88948.010   Durbin-Watson:                   1.907
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           169880.705
Skew:                          -1.586   Prob(JB):                         0.00
Kurtosis:                       4.072   Cond. No.                     3.08e+05
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The condition number is large, 3.08e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [100]:
sm.WLS(cps_c_hours.hours,
       cps_c_hours[['days_sc_has_k6_female',  # Beta of interest.
                    'days_sc_female', 'days_sc_has_k6', 'has_k6_female',
                    'const', 'apr2020', 'has_k6',
                    'days_since_2000', 'female', 'age', 'age2']],
       weights=cps_c_hours.w, missing='drop').fit(
    cov_type='cluster', cov_kwds={'groups': cps_c_hours.COUNTY}).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  hours   R-squared:                       0.040
Model:                            WLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     157.9
Date:                Sat, 30 May 2020   Prob (F-statistic):           1.44e-80
Time:                        18:07:37   Log-Likelihood:            -1.1114e+06
No. Observations:              283593   AIC:                         2.223e+06
Df Residuals:                  283582   BIC:                         2.223e+06
Df Model:                          10                                         
Covariance Type:              cluster                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
days_sc_has_k6_female     0.0294      0.016      1.831      0.067      -0.002       0.061
days_sc_female            0.0060      0.012      0.507      0.612      -0.017       0.029
days_sc_has_k6           -0.0228      0.012     -1.915      0.056      -0.046       0.001
has_k6_female            -2.8974      0.185    -15.661      0.000      -3.260      -2.535
const                    31.2414      1.344     23.246      0.000      28.607      33.876
apr2020                  -1.7005      0.289     -5.893      0.000      -2.266      -1.135
has_k6                    0.8386      0.124      6.783      0.000       0.596       1.081
days_since_2000           0.0001      0.000      0.788      0.430      -0.000       0.000
female                   -2.7902      0.116    -24.006      0.000      -3.018      -2.562
age                       0.4556      0.052      8.785      0.000       0.354       0.557
age2                     -0.0049      0.001     -7.663      0.000      -0.006      -0.004
==============================================================================
Omnibus:                    44065.780   Durbin-Watson:                   1.903
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           554585.306
Skew:                           0.342   Prob(JB):                         0.00
Kurtosis:                       9.816   Cond. No.                     3.07e+05
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The condition number is large, 3.07e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
PanelOLS()